# Labeling the amazon reviews with Afinn sentiment analysis.

(...)

- Sentiment score from all of the words in Amazon reviews.
- Sentiment score from only the most frequent words in Amazon reviews.

### Sentiment score from all of the words in Amazon reviews.

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import gzip
import nltk
import gensim
import collections
import random
from afinn import Afinn
%matplotlib inline

In [24]:
# # This cell is for making cleaned_reviews.json file. If you already have this, you don't need to run this cell.
# # If you want to run, to remove all # at once, "command + a" -> "command + /".

# #AMAZON DATA
# REVIEWS_PATH = "../data/02_processed/reviews_Grocery_and_Gourmet_Food.json.gz"
# META_PATH = "../data/02_processed/meta_Grocery_and_Gourmet_Food.json.gz"
# #FDA DATA
# path_press_released = "/Users/tajimakeijiro/Desktop/amazon_project/Amazon-Reviews-Infectious-Disease/data/02_processed/FDA_press_released_2011_2019.csv"
# path_enforced = "/Users/tajimakeijiro/Desktop/amazon_project/Amazon-Reviews-Infectious-Disease/data/02_processed/FDA_enforcements_2012-06_to_2019-10.csv"

# def sanitize(path, outpath):
#     """Converts a given compressed json to strict json and writes it in a new file

#     Parameters
#     ----------
#     path : str
#     The file location of the gzip-compressed json file
#     outpath : str
#     The path to the desired output file location 

#     """
#     g = gzip.open(path, 'r')

#     out = open(outpath, 'w')

#     for l in g:
#         out.write(json.dumps(eval(l)) + '\n')
#     out.close()
    
# META_OUTPATH = "cleaned_meta.json"
# REVIEWS_OUTPATH = "cleaned_reviews.json"

# sanitize(META_PATH, META_OUTPATH)
# sanitize(REVIEWS_PATH, REVIEWS_OUTPATH)

In [18]:
#Download stopwords and wordnet for lemmatization (only need to be executed once)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tajimakeijiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tajimakeijiro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
# Use a json file made in the notebook "Milestone2.ipynb".
REVIEWS_PATH = "./cleaned_reviews.json"

Start by importing the data :

In [22]:
reviews = pd.read_json(REVIEWS_PATH, lines=True)

#TBD: Which columns to keep/remove
reviews = reviews.drop(columns=['reviewerName', 'helpful', 'reviewTime'])

#Convert the utc timestamp to readable dates
reviews['unixReviewTime'] = pd.to_datetime(reviews['unixReviewTime'],unit='s')

reviews.head()

,reviewerID,asin,reviewText,overall,summary,unixReviewTime
0,A1ZQZ8RJS1XVTX,0657745316,"No sugar, no GMO garbage, no fillers that come...",5,Best vanilla I've ever had,2013-10-11
1,A31W38VGZAUUM4,0700026444,"This is my absolute, undisputed favorite tea r...",5,Terrific Tea!,2012-12-06
2,A3I0AV0UJX5OH0,1403796890,I ordered spongbob slippers and I got John Cen...,1,grrrrrrr,2013-12-02
3,A3QAAOLIXKV383,1403796890,The cart is fine and works for the purpose for...,3,Storage on Wheels Cart,2011-06-12
4,AB1A5EGHHVA9M,141278509X,This product by Archer Farms is the best drink...,5,The best drink mix,2012-03-24


Create a function to process the reviews using the nltk library :
* We tokenize the sentence,
* remove any potential stop words,
* remove tokens containing only punctuations (such as '!!!', '...', etc.. which where quite common),
* remove words below a given length,
* stem the words to have them all represented in a standardized way. 

In [25]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

stop_words = set(stopwords.words('english')) 

# Here we don't need stemming, because Afinn has same score for the before stemmed ones.
# stemmer = PorterStemmer()

def process_text(sentence):
    token_words = nltk.word_tokenize(sentence)
    no_stopwords = [word.lower() for word in token_words if word not in stop_words and not \
                    all(c in string.punctuation for c in word) and not len(word) < 2]
    # return [stemmer.stem(word) for word in no_stopwords]
    return [(word) for word in no_stopwords]

print(process_text('I ordered spongbob slippers and I got John'))

['ordered', 'spongbob', 'slippers', 'got', 'john']


We add a new column to our dataframe containing the processed reviewText (notice that we only keep reviews with a low score, under the fair assumption that reviews exposing health issues would have a low rating).

In [26]:
stemmed = reviews.copy()
stemmed = stemmed[stemmed['overall'] < 3]
stemmed['reviewStemmed'] = stemmed['reviewText'].apply(lambda x : process_text(x))

stemmed.head()

,reviewerID,asin,reviewText,overall,summary,unixReviewTime,reviewStemmed
2,A3I0AV0UJX5OH0,1403796890,I ordered spongbob slippers and I got John Cen...,1,grrrrrrr,2013-12-02,"[ordered, spongbob, slippers, got, john, cena,..."
5,A3DTB6RVENLQ9Q,1453060375,Don't buy this item - rip off at this price. ...,1,Oops. Made a mistake and ordered this. I mis...,2013-03-03,"[do, n't, buy, item, rip, price, my, bad, mist..."
46,A3KJ9TZ2HLL7SA,5901002482,I wrote an earlier scathing review of this pro...,1,Packaging problem,2012-11-28,"[wrote, earlier, scathing, review, product, wh..."
48,ACEL2LY99MAB0,6162362183,I read the reviews before I bought it. It got ...,2,Very disappointed.,2014-04-21,"[read, reviews, bought, it, got, excited, revi..."
61,A2F3CK8F9VIFPL,616719923X,I bought it because i like green tea but the t...,1,Yuck,2013-07-29,"[bought, like, green, tea, taste, bad, came, m..."


Here, we simply store the dataframe in a pickle for later usage.

In [27]:
# stemmed.to_pickle("reviews_tokened_tenth")

In [28]:
afinn = Afinn()

# This cell takes long
stemmed['afinnWords'] = stemmed['reviewText'].apply(lambda x : afinn.find_all(x))
stemmed['afinnScores'] = stemmed['reviewText'].apply(lambda x : afinn.scores_with_pattern(x))
stemmed['afinnTotalScore'] = stemmed['reviewText'].apply(lambda x : afinn.score_with_pattern(x))
stemmed['afinnWordsLen'] = stemmed['afinnWords'].apply(lambda x : len(x))

In [30]:
# Define a function that flatten one column into a list
def flatten(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [31]:
l_stemmed = stemmed['afinnWords'] .values.tolist()
l_stemmed = flatten(l_stemmed)
print(len(set(l_stemmed)))

2688


In [32]:
stemmed['afinnMeanScore'] = stemmed['afinnTotalScore']/stemmed['afinnWordsLen']

In [33]:
stemmed.head()

,reviewerID,asin,reviewText,overall,summary,unixReviewTime,reviewStemmed,afinnWords,afinnScores,afinnTotalScore,afinnWordsLen,afinnMeanScore
2,A3I0AV0UJX5OH0,1403796890,I ordered spongbob slippers and I got John Cen...,1,grrrrrrr,2013-12-02,"[ordered, spongbob, slippers, got, john, cena,...",[happy],[3],3.0,1,3.000000
5,A3DTB6RVENLQ9Q,1453060375,Don't buy this item - rip off at this price. ...,1,Oops. Made a mistake and ordered this. I mis...,2013-03-03,"[do, n't, buy, item, rip, price, my, bad, mist...","[bad, mistake, pay, pay]","[-3, -2, -1, -1]",-7.0,4,-1.750000
46,A3KJ9TZ2HLL7SA,5901002482,I wrote an earlier scathing review of this pro...,1,Packaging problem,2012-11-28,"[wrote, earlier, scathing, review, product, wh...","[harsh, apologize, disappointed, protect, hope...","[-2, -1, -2, 1, 2, -2]",-4.0,6,-0.666667
48,ACEL2LY99MAB0,6162362183,I read the reviews before I bought it. It got ...,2,Very disappointed.,2014-04-21,"[read, reviews, bought, it, got, excited, revi...","[excited, good, destroyed, mad, disappointed, ...","[3, 3, -3, -3, -2, 3]",1.0,6,0.166667
61,A2F3CK8F9VIFPL,616719923X,I bought it because i like green tea but the t...,1,Yuck,2013-07-29,"[bought, like, green, tea, taste, bad, came, m...","[like, bad]","[2, -3]",-1.0,2,-0.500000


In [85]:
l_words = stemmed['afinnWords'].values.tolist()
l_words = flatten(l_words)
l_scores = [afinn.scores_with_pattern(x) for x in l_words]
words_scores = list(zip(l_words, l_scores)) 
positive_words = set([word for word, score in words_scores if score[0] > 0])
negative_words = set([word for word, score in words_scores if score[0] < 0])

In [43]:
stemmed.groupby(by=['afinnWordsLen']).describe()

overall                                               \
                 count      mean       std  min  25%  50%  75%  max   
afinnWordsLen                                                         
0               7287.0  1.346370  0.475846  1.0  1.0  1.0  2.0  2.0   
1              19320.0  1.381832  0.485848  1.0  1.0  1.0  2.0  2.0   
2              26116.0  1.384515  0.486490  1.0  1.0  1.0  2.0  2.0   
3              25653.0  1.381008  0.485644  1.0  1.0  1.0  2.0  2.0   
4              20578.0  1.375498  0.484263  1.0  1.0  1.0  2.0  2.0   
...                ...       ...       ...  ...  ...  ...  ...  ...   
140                1.0  2.000000       NaN  2.0  2.0  2.0  2.0  2.0   
142                2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0   
180                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0   
202                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0   
247                1.0  2.000000       NaN  2.0  2.0  2.0  2.0  2.0   

              afinnTotalScore              ...               afinnMeanScore  \
                        count        mean  ...    75%    max          count   
afinnWordsLen                              ...                                
0                      7287.0    0.000000  ...    0.0    0.0            0.0   
1                     19320.0    0.389337  ...    2.0    5.0        19320.0   
2                     26116.0    0.798399  ...    4.0    9.0        26116.0   
3                     25653.0    1.143219  ...    4.0   12.0        25653.0   
4                     20578.0    1.550296  ...    5.0   14.0        20578.0   
...                       ...         ...  ...    ...    ...            ...   
140                       1.0   -1.000000  ...   -1.0   -1.0            1.0   
142                       2.0  129.000000  ...  160.5  192.0            2.0   
180                       1.0   -7.000000  ...   -7.0   -7.0            1.0   
202                       1.0   52.000000  ...   52.0   52.0            1.0   
247                       1.0  400.000000  ...  400.0  400.0            1.0   

                                                                           \
                   mean       std       min       25%       50%       75%   
afinnWordsLen                                                               
0                   NaN       NaN       NaN       NaN       NaN       NaN   
1              0.389337  2.176076 -4.000000 -2.000000  1.000000  2.000000   
2              0.399200  1.569018 -4.000000 -0.500000  0.500000  2.000000   
3              0.381073  1.310980 -3.333333 -0.666667  0.333333  1.333333   
4              0.387574  1.164667 -3.250000 -0.500000  0.500000  1.250000   
...                 ...       ...       ...       ...       ...       ...   
140           -0.007143       NaN -0.007143 -0.007143 -0.007143 -0.007143   
142            0.908451  0.627433  0.464789  0.686620  0.908451  1.130282   
180           -0.038889       NaN -0.038889 -0.038889 -0.038889 -0.038889   
202            0.257426       NaN  0.257426  0.257426  0.257426  0.257426   
247            1.619433       NaN  1.619433  1.619433  1.619433  1.619433   

                         
                    max  
afinnWordsLen            
0                   NaN  
1              5.000000  
2              4.500000  
3              4.000000  
4              3.500000  
...                 ...  
140           -0.007143  
142            1.352113  
180           -0.038889  
202            0.257426  
247            1.619433  

[93 rows x 24 columns]

### Sentiment score from only the most frequent words in Amazon reviews.

In [44]:
def flatten(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [45]:
l_stemmed = stemmed['reviewStemmed'].values.tolist()
l_stemmed = flatten(l_stemmed)

In [46]:
frequentWords = collections.Counter(l_stemmed).most_common()[0:50000]
frequentWords = [word for word, count in frequentWords]

In [47]:
stemmed_frequent = stemmed.copy()
stemmed_frequent['afinnWords'] = stemmed['afinnWords'].apply(lambda x : [i for i in x if i in frequentWords])
stemmed_frequent['afinnScores'] = stemmed_frequent['reviewText'].apply(lambda x : afinn.scores_with_pattern(x))
stemmed_frequent['afinnTotalScore'] = stemmed_frequent['reviewText'].apply(lambda x : afinn.score_with_pattern(x))
stemmed_frequent['afinnWordsLen'] = stemmed_frequent['afinnWords'].apply(lambda x : len(x))

In [48]:
stemmed_frequent['afinnMeanScore'] = stemmed_frequent['afinnTotalScore']/stemmed_frequent['afinnWordsLen']

In [49]:
stemmed_frequent.head()

,reviewerID,asin,reviewText,overall,summary,unixReviewTime,reviewStemmed,afinnWords,afinnScores,afinnTotalScore,afinnWordsLen,afinnMeanScore
2,A3I0AV0UJX5OH0,1403796890,I ordered spongbob slippers and I got John Cen...,1,grrrrrrr,2013-12-02,"[ordered, spongbob, slippers, got, john, cena,...",[happy],[3],3.0,1,3.000000
5,A3DTB6RVENLQ9Q,1453060375,Don't buy this item - rip off at this price. ...,1,Oops. Made a mistake and ordered this. I mis...,2013-03-03,"[do, n't, buy, item, rip, price, my, bad, mist...","[bad, mistake, pay, pay]","[-3, -2, -1, -1]",-7.0,4,-1.750000
46,A3KJ9TZ2HLL7SA,5901002482,I wrote an earlier scathing review of this pro...,1,Packaging problem,2012-11-28,"[wrote, earlier, scathing, review, product, wh...","[harsh, apologize, disappointed, protect, hope...","[-2, -1, -2, 1, 2, -2]",-4.0,6,-0.666667
48,ACEL2LY99MAB0,6162362183,I read the reviews before I bought it. It got ...,2,Very disappointed.,2014-04-21,"[read, reviews, bought, it, got, excited, revi...","[excited, good, destroyed, mad, disappointed, ...","[3, 3, -3, -3, -2, 3]",1.0,6,0.166667
61,A2F3CK8F9VIFPL,616719923X,I bought it because i like green tea but the t...,1,Yuck,2013-07-29,"[bought, like, green, tea, taste, bad, came, m...","[like, bad]","[2, -3]",-1.0,2,-0.500000


In [86]:
l_frequent_words = stemmed_frequent['afinnWords'].values.tolist()
l_frequent_words = flatten(l_frequent_words)
l_frequent_scores = [afinn.scores_with_pattern(x) for x in l_frequent_words]
words_scores = list(zip(l_frequent_words, l_frequent_scores)) 
positive_frequent_words = set([word for word, score in words_scores if score[0] > 0])
negative_frequent_words = set([word for word, score in words_scores if score[0] < 0])

In [50]:
stemmed.groupby(by=['afinnWordsLen']).describe()

overall                                               \
                 count      mean       std  min  25%  50%  75%  max   
afinnWordsLen                                                         
0               7287.0  1.346370  0.475846  1.0  1.0  1.0  2.0  2.0   
1              19320.0  1.381832  0.485848  1.0  1.0  1.0  2.0  2.0   
2              26116.0  1.384515  0.486490  1.0  1.0  1.0  2.0  2.0   
3              25653.0  1.381008  0.485644  1.0  1.0  1.0  2.0  2.0   
4              20578.0  1.375498  0.484263  1.0  1.0  1.0  2.0  2.0   
...                ...       ...       ...  ...  ...  ...  ...  ...   
140                1.0  2.000000       NaN  2.0  2.0  2.0  2.0  2.0   
142                2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0   
180                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0   
202                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0   
247                1.0  2.000000       NaN  2.0  2.0  2.0  2.0  2.0   

              afinnTotalScore              ...               afinnMeanScore  \
                        count        mean  ...    75%    max          count   
afinnWordsLen                              ...                                
0                      7287.0    0.000000  ...    0.0    0.0            0.0   
1                     19320.0    0.389337  ...    2.0    5.0        19320.0   
2                     26116.0    0.798399  ...    4.0    9.0        26116.0   
3                     25653.0    1.143219  ...    4.0   12.0        25653.0   
4                     20578.0    1.550296  ...    5.0   14.0        20578.0   
...                       ...         ...  ...    ...    ...            ...   
140                       1.0   -1.000000  ...   -1.0   -1.0            1.0   
142                       2.0  129.000000  ...  160.5  192.0            2.0   
180                       1.0   -7.000000  ...   -7.0   -7.0            1.0   
202                       1.0   52.000000  ...   52.0   52.0            1.0   
247                       1.0  400.000000  ...  400.0  400.0            1.0   

                                                                           \
                   mean       std       min       25%       50%       75%   
afinnWordsLen                                                               
0                   NaN       NaN       NaN       NaN       NaN       NaN   
1              0.389337  2.176076 -4.000000 -2.000000  1.000000  2.000000   
2              0.399200  1.569018 -4.000000 -0.500000  0.500000  2.000000   
3              0.381073  1.310980 -3.333333 -0.666667  0.333333  1.333333   
4              0.387574  1.164667 -3.250000 -0.500000  0.500000  1.250000   
...                 ...       ...       ...       ...       ...       ...   
140           -0.007143       NaN -0.007143 -0.007143 -0.007143 -0.007143   
142            0.908451  0.627433  0.464789  0.686620  0.908451  1.130282   
180           -0.038889       NaN -0.038889 -0.038889 -0.038889 -0.038889   
202            0.257426       NaN  0.257426  0.257426  0.257426  0.257426   
247            1.619433       NaN  1.619433  1.619433  1.619433  1.619433   

                         
                    max  
afinnWordsLen            
0                   NaN  
1              5.000000  
2              4.500000  
3              4.000000  
4              3.500000  
...                 ...  
140           -0.007143  
142            1.352113  
180           -0.038889  
202            0.257426  
247            1.619433  

[93 rows x 24 columns]

### For UI, make dislike words lists.
From Amazon reviews, we extracted the words which have minus score in Afinn as dislike words.

In [87]:
# Like and Dislike words list from all of the reviews.
print(random.sample(positive_words, 5))
print(random.sample(negative_words, 5))

['bless', 'interests', 'careful', 'motivation', 'jewel']
['denied', 'diseases', 'mad', 'deceit', 'suicides']


In [88]:
print(random.sample(positive_frequent_words, 5))
print(random.sample(negative_frequent_words, 5))

['joke', 'vitality', 'justice', 'positively', 'compliment']
['hates', 'disapproval', 'chaos', 'stabbed', 'disruption']


In [89]:
df = pd.DataFrame([positive_words, negative_words, positive_frequent_words, negative_frequent_words], index = ["Positive_words", "Negative_words", "Positive_words_only_frequent", "Negative_words_only_frequent"])

In [90]:
df

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703
Positive_words,defenders,admires,donates,comfortable,awesome,adequate,leading,endorses,fearless,immune,...,None,None,None,None,None,None,None,None,None,None
Negative_words,frustrated,sinful,undermines,offender,helpless,misused,exhausted,refused,infesting,apathy,...,obstructs,worsens,cried,protest,inoperative,fraudulence,worries,cruel,chastise,unethical
Positive_words_only_frequent,defenders,donates,comfortable,awesome,adequate,leading,endorses,immune,trusted,reach,...,None,None,None,None,None,None,None,None,None,None
Negative_words_only_frequent,frustrated,sinful,offender,helpless,exhausted,refused,infesting,apathy,brutally,atrocity,...,None,None,None,None,None,None,None,None,None,None


In [91]:
df.to_csv('../data/02_processed/Like_and_dislike_words.csv')